# code is for extraction of side effect from drugs.com website

In [10]:
from bs4 import BeautifulSoup as B
from selenium import webdriver
import pandas as pd
import time
import re
from nltk.corpus import stopwords
import seaborn as sns
import matplotlib.pyplot as plt
from selenium.webdriver.support.ui import Select
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
wnl=WordNetLemmatizer()
websitelist=[]
userlist=[]

In [2]:
#driver=webdriver.Chrome()
driver=webdriver.Firefox()

In [3]:
url='https://www.drugs.com/drug_information.html'
driver.get(url)
soup=B(driver.page_source,'html.parser')

In [4]:
link=soup.find_all('a')

In [5]:
a=link[3:29]

In [6]:
websitelist=[]
for l in a:
    v=l.get('href')
    websitelist.append('https://www.drugs.com'+v)

In [7]:
l=pd.DataFrame(websitelist,columns=['Website'])

In [8]:
sideeffect=[]
def sideeffects(url):
    driver.get(url)
    soup=B(driver.page_source,'html.parser')
    h=soup.find_all('ul',{'class':"list-length-short list-type-paragraph"})
    if h:
        for i in h:
            a=i.text.split('\n')[1:-1]
            for k in a:
                sideeffect.append(k)

In [9]:
def get_urls(a):
    url=a
    driver.get(url)
    soup=B(driver.page_source,'html.parser')
    link=soup.find_all('ul',{'class':'ddc-list-column-2'})
    if link:
        pattern=re.compile(r'href="(/mtm)?(/cons)?(/cdi)?(/npc)?(/pro)?(/ppa)?([A-za-z/.]+)"')
        matches1=pattern.finditer(str(link[0]))
        time.sleep(5)
        for match in matches1:
            c=str(match[7][:-5])
            b='https://www.drugs.com/'+'sfx'+ c +'-side-effects.html'
            sideeffects(b)

    

In [10]:

def get_page(d):
    url=d
    driver.get(url)
    soup=B(driver.page_source,'html.parser')
    link=soup.find_all('div',{'class':'ddc-box ddc-mgb-2 paging-list-wrap'})
    if link:
        pattern=re.compile(r'href="([A-za-z/.]+)"')
        matches=pattern.finditer(str(link[0]))
        for match in matches:
            a='https://www.drugs.com'+str(match[1])
            get_urls(a)

    
    
    

In [233]:
for o in l.Website:
    get_page(o)

In [236]:
sideeffect

['Abdominal or stomach pain',
 'cough',
 'diarrhea',
 'difficult or labored breathing',
 'fever',
 'headache',
 'joint or muscle pain',
 'nausea',
 'numbness or tingling of the hands, feet, or face',
 'redness and soreness of the eyes',
 'skin rash',
 'sore throat',
 'sores in the mouth',
 'swelling of the feet or lower legs',
 'unusual feeling of discomfort or illness',
 'unusual tiredness',
 'vomiting',
 'Blistering, peeling, or loosening of the skin',
 'chest pain or discomfort',
 'chills',
 'dark urine',
 'itching',
 'light-colored stools',
 'pain or discomfort in the arms, jaw, back, or neck',
 'red, irritated eyes',
 'red skin lesions, often with a purple center',
 'sores, ulcers, or white spots in the mouth or on the lips',
 'sweating',
 'unusual weakness',
 'upper right abdominal or stomach pain',
 'yellow eyes and skin',
 'Back pain',
 'bladder pain',
 'bloody or cloudy urine',
 'body aches or pain',
 'chills',
 'cough',
 'cough producing mucus',
 'difficult, burning, or painf

In [237]:
len(sideeffect)

75419

In [238]:
a=pd.DataFrame(sideeffect,columns=['sideeffect'])

In [242]:
a=a.drop_duplicates()

In [244]:
a.to_csv('side-effect.csv',index=False)

# Extracting words from side-effect.csv(above extracted file)

In [11]:
a=pd.read_csv('side-effect.csv')
a['sideeffect']=a.sideeffect.str.lower()
a['sideeffect']=a.sideeffect.apply(lambda x: re.sub(r'[^\w\s]+','',x))
a['sideeffect']=a['sideeffect'].apply(lambda x:[wnl.lemmatize(word) for word in x.split()])
a['sideeffect']=a.sideeffect.apply(lambda x: [word for word in x if word not in stop_words])
b=list(a['sideeffect'])
s=[]
for c in b:
    for d in c:
        s.append(d)


In [14]:
len(s)

13766

In [15]:
b=pd.DataFrame(s,columns=['sideeffect'])

In [20]:
b=b.drop_duplicates().reset_index(drop=True)

In [21]:
b

,sideeffect
0,abdominal
1,stomach
2,pain
3,cough
4,diarrhea
...,...
1233,combative
1234,ill
1235,slowness
1236,forth


In [73]:
b.to_csv('side-effect_S.csv',index=False)

# conditonal probability database creation function 

In [ ]:
# Create database for conditional probility(one time)
def conditional_pro(sen,u):
    p=pd.crosstab(u['sideeffect'],u['sideeffect'],0,aggfunc=np.mean)
    p=p.replace(np.nan,0)
    ind=list(p.index)
    col=list(p.columns)
    for i in range(len(ind)):
        f=[]
        for j in range(len(ind)):
            k=0
            for z in range(sen.shape[0]):
                c=sen[z].split()
                if(ind[i] in c)&(col[j] in c):
                    k=k+1
            p.loc[p.index==ind[i],col[j]]=k
        print(i,j)
    return(p)   

k=pd.read_csv('side-effect_s.csv')
o=pd.read_csv('side-effect.csv')
t=conditional_pro(o['sideeffect'],k)
t.to_csv('conditional_probability.csv')